In [60]:
import numpy as np
import time

In [61]:
"""
height h of each trapezoid is founb by doing b-a/n as all the trapezoids are equally distributed
area under the curve is found by summing up area of each trapezoid by directly applying the area of trapezoid formula.
"""
def py_trapz(f, a, b, n):
    area = 0
    h = (b - a)/n
    for i in range(n):
        x0 = a + i * h      
        x1 = a + (i + 1) * h 
        area += (f(x0) + f(x1)) * 0.5 * h  
    return area

In [62]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [65]:
%%cython
"""
cython's static tpying is done to improve speed by converting python code to c and then again the same method used in py_trapz is used to find area.
"""
cdef double cy_trapz(object f, double a, double b, int n):
    cdef double h = (b - a) / n
    cdef double result = 0.5 * (f(a) + f(b))
    cdef int i

    for i in range(1, n):
        result += f(a + i * h)
    return result * h

def cython_trapz_test(f, a, b, n):
    return cy_trapz(f, a, b, n)

In [66]:
#using in-built numpy function
def numpy_trapz(f, a, b, n):
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return np.trapz(y, x)

In [67]:
#all the 4 functions are defined seperately
def x2(x): return x**2
def sinx(x): return np.sin(x)
def ex(x): return np.exp(x)
def inv(x): return 1/x

In [86]:
# Function to calculate the exact integral value for error calculation
def exact_integral(f, a, b):
    if f == x2:
        return (b**3 - a**3) / 3  # ∫x^2 dx from a to b = (b^3 - a^3) / 3
    if f == sinx:
        return 1  # ∫sin(x) dx from 0 to π = 2
    if f == ex:
        return np.exp(b) - np.exp(a)  # ∫e^x dx from a to b = e^b - e^a
    if f == inv:
        return np.log(b) - np.log(a)  # ∫1/x dx from a to b = ln(b) - ln(a)
    
    
#function to print out the results
def final(f, a, b, n):
    exact_value = exact_integral(f, a, b)
    start = time.time()
    result_python = py_trapz(f, a, b, n)
    end = time.time()
    error_python = abs(exact_value - result_python)
    print(f"Python Result: {result_python}, Time: {end - start:.6f}s, Error: {error_python}")
    
    start = time.time()
    result_cython = cython_trapz_test(f, a, b, n) 
    end = time.time()
    error_cython = abs(exact_value - result_cython)
    print(f"Cython Result: {result_cython}, Time: {end - start:.6f}s, Error: {error_cython}")
    
    start = time.time()
    result_numpy = numpy_trapz(f, a, b, n)
    end = time.time()
    error_numpy = abs(exact_value - result_numpy)
    print(f"NumPy Result: {result_numpy}, Time: {end - start:.6f}s, Error: {error_numpy}")
    
    
final(x2, 0, 1, 1000)
final(sinx, 0, np.pi, 1000)
final(ex, 0, 1, 1000)
final(inv, 1, 2, 1000)

Python Result: 0.33333349999999984, Time: 0.000390s, Error: 1.6666666652342954e-07
Cython Result: 0.33333349999999995, Time: 0.000124s, Error: 1.6666666663445184e-07
NumPy Result: 0.3333335, Time: 0.000254s, Error: 1.66666666689963e-07
Python Result: 1.999998355065662, Time: 0.001727s, Error: 0.9999983550656619
Cython Result: 1.9999983550656624, Time: 0.000696s, Error: 0.9999983550656624
NumPy Result: 1.9999983550656628, Time: 0.001002s, Error: 0.9999983550656628
Python Result: 1.718281971649196, Time: 0.001722s, Error: 1.431901508475164e-07
Cython Result: 1.7182819716491962, Time: 0.000721s, Error: 1.43190151069561e-07
NumPy Result: 1.718281971649195, Time: 0.000137s, Error: 1.4319014995933799e-07
Python Result: 0.6931472430599359, Time: 0.000304s, Error: 6.249999062735156e-08
Cython Result: 0.6931472430599374, Time: 0.000089s, Error: 6.24999920706415e-08
NumPy Result: 0.6931472430599375, Time: 0.000130s, Error: 6.24999921816638e-08


In [71]:
#a performance test by integrating f(x) = x^2 from 0 to 10 with 10 million trapezoids
final(x2, 0, 10, 10000000)

Python Result: 333.3333333333152, Time: 3.700722s
Cython Result: 333.33333333334724, Time: 1.128303s
NumPy Result: 333.3333333333349, Time: 0.141199s
